# Introduction to CKKS

## Introduction to Homomorphic Encryption

### Context

Homomorphic encryption (HE) is an encryption scheme, which allows data owners to encrypt their data, and let a third party perform computations on it, without knowing what is the underlying data. The result of the computations on encrypted data can then be sent back to the data owner, which will be the only one able to decrypt the encrypted result.

HE enables computation to be performed on encrypted data, while preserving the decryption output, as it leverages homomorphic properties of the encryption and the decryption. This means that addition and multiplication can be done on encrypted data, and the decryption output will be as if those operations were done directly on the unencrypted data.

More formally, a ring homomorphism $h$ between two rings $R$ and $R'$, follows those two properties : 
$$h(x + y) = h(x) + h(y)$$
$$h(x * y) = h(x) * h(y)$$

This means that if we have an encryption homomorphism $e$, a decryption homomorphism $d$, such that $d(e(x)) = x$, and a function $f$, which is a composition of additions and multiplications, then we can have the following scenario :

- The user encrypts their data $x$ using $e$, and sends $e(x)$ to an untrusted third party.
- The third party performs computations $f$ on the encrypted $e(x)$. Because $e$ is an homomorphism, we have that $f(ex(x)) = e(fx)$. Then the third party sends the data back to the user.
- Finally the user decrypts the output, obtaining then $d(e(f(x))) = f(x)$, without exposing her data directly to the untrusted third party.

### The CKKS scheme

![Overview CKKS](images/overview_ckks.PNG)
<center>High level view of CKKS.</center>

We will focus now on the CKKS scheme, which allows homomorphic operations on complex, thus real, values. The figure above provides a high level view of how CKKS works :

- The user generates a secret and public key beforehand.
- The user then first encodes a complex vector, on which computation will be done, into a plaintext polynomial.
- This plaintext polynomial is encrypted into a ciphertext made of two polynomials using the public key.
- Computations are then performed on the ciphertext by a third party.
- The results are decrypted, using the private key, into a plaintext polynomial.
- Finally the user can decode and read the values.

#### Notations

When using the CKKS scheme, one has to choose two important parameters :

- $q$ the modulus used in the computation. Most computations will be done modulo $q$. 
- $N$ the degree of the polynomial modulus. This will help us define the plaintext and ciphertext space, as we will use  $\mathcal{R} = \mathbb{Z}[X]/(X^N + 1)$ as the plaintext space, and $\mathcal{R_q} = \mathbb{Z}_q[X]/(X^N + 1)$ as the ciphertext space. 

The important thing to remember is that the higher $N$ implies higher security, but also more computations. On the other hand, the higher $q$, the more multiplications we will be able to perform, but it makes the system weaker.

We will see in practice how to choose the right parameters.

#### Encoding and decoding

Because the plaintext and ciphertext space are polynomial spaces, we must first encode our input, which will be a complex vector $z \in \mathbb{C}^{N/2}$, into a polynomial $m(X) \in \mathcal{R}$. This is done by polynomial interpolation on the roots of $X^N + 1$. 

One important thing to consider, is that when a vector is encoded, we will have to choose a scale $\Delta > 0$, which will represent how precise we want our computation to be. The caveat is that the higher $\Delta$, the bigger $q$, and therefore we need to have higher $N$ to compensate to keep the system secure.

Finally, decoding will be just evaluatinga polynomial $m(X) \in \mathcal{R}$ on the same roots defined before.

#### Encryption and decryption

CKKS is a public encryption scheme, where the user will generate a secret key $s_k$, and a public key $p_k$. 

The idea is that the user will sample $a$ a mask, $s_k$ a secret key, and $e$ a small noise. $s_k$ will be used for decryption, while $a$ and $e$ are used to generate a public key. The public key $p_k$ will then be :

$p_k = (-a s_k + e, a) = (b, a)$, with $b = - a s_k + e$.

Note that all the quantities above are polynomials.

The security of CKKS is based on the fact that is extremely hard to find $s_k$ from the equation $b = - a s_k + e$. This problem is called the Ring Learning With Error, and is quantum secure, which means not crackable by quantum computers.

Using the public key $p_k$, a user can encrypt a plaintext $m$ using the public key $p_k$ by outputing the ciphertext : 
$c = (b + m, a) = (c_0, c_1)$

Then to decrypt the message, one simply needs to use the secret key and compute : 
$c_0 + s_k c_1 = b + m + s_k a = -a s_k + e + m + s_k a = m + e$, because $b = -a s_k + e$.

We see that after decrypting we obtain $m +e$, with $m$ our original message, and $e$ a small error polynomial. So we do not exactly get the original $m$ but one that is slightly disturbed by a small noise $e$, but if the noise is small enough we should have $m + e \approx m$.

#### Homomorphic operations

Additions between ciphertext will be pretty straightforward, we just need to add together the polynomials of each ciphertext.

Multiplications are also possible, but they come with complexity. Without going into too much detail, there are two problems that need to be solved when doing multiplication : 

- Instead of needing two polynomials to decrypt using the secret key, three polynomials are required to decrypt a ciphertext after a multiplication. This means that the size has increased, and to keep the size of the ciphertext constant, we must relinearize it to go from three polynomials to only two.
- The scale will have changed, as if we multiply two ciphertexts, one encrypting $\Delta x$, and another one $\Delta y$, then the output will be $\Delta^2 xy$, therefore the scale has increased, and if we do nothing the encrypted value will overflow and we will not be able to recover it. That's why we need to rescale the output, using one of the primes we will define in $q$, but in doing so, we reduce the number of multiplications left.

## Deep dive in the code

Now enough talk, let's see how all of this works out in practice ! 

We will be using the [Microsoft Research library SEAL](https://github.com/Microsoft/SEAL) to play with the CKKS scheme. Because SEAL is a C++ library, we will use the Python wrapper developped by [OpenMined](https://www.openmined.org/), [TenSEAL](https://github.com/OpenMined/TenSEAL), to interact with SEAL.

### Setup

In [1]:
import tenseal.sealapi as seal

First we will set the parameters needed by SEAL for the context. 

The rule of thumb to choose your parameters is : 
- How many multiplications will you do ?
- How precise does your calculation need to be ?
- How big do you think your output will be ?

This will help choose the moduli $q$, which is the product of primes that we will use for rescaling.

In our case, let's say we want to do 2 multiplications, use 40 bits for precision, i.e. results are precise up to $\Delta^{-1} = 2^{-40} \approx 10^{-13}$. We expect our output to be rather big, therefore we will allow ourselves to have 20 bits to encode the integral part of the output.

Once you have all this information, you can set $q$ using the following strategy : 
- Choose a 60-bit prime as the first prime in coeff_modulus. This will give the highest precision when decrypting
- Choose another 60-bit prime as the last element of coeff_modulus, as this will be used as the special prime and should be as large as the largest of the other primes.
- Choose the intermediate primes to be close to each other.

In that case, we will have therefore [60, 40, 40, 60]. The first 60 means we want as much precision as possible when decrypting, the two 40 means we will use two multiplication and we want 40 bits of scale, and the last 60 is just a parameter that must be equal to the largest prime used.

Once we have chosen the moduli, we just need to sum it and find the appropriate $N$ to ensure maximum security and efficiency according to this table : 

<table>
    <thead>
        <tr>
            <td>$N$</td>
            <td>Maximum bits in $q$.</td>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td>1024</td>
            <td>27</td>
        </tr>
        <tr>
            <td>2048</td>
            <td>54</td>
        </tr>
        <tr>
            <td>4096</td>
            <td>109</td>
        </tr>
        <tr>
            <td>8192</td>
            <td>218</td>
        </tr>
        <tr>
            <td>16384</td>
            <td>438</td>
        </tr>
        <tr>
            <td>32768</td>
            <td>881</td>
        </tr>
    </tbody>
</table>

Here because we have 60 + 40 + 40 + 60 = 200 bits used in $q$, we will choose $N = 8192$. 

In [2]:
poly_modulus_degree = 8192
moduli = [60, 40, 40, 60]
PRECISION_BITS = 40
scale = pow(2.0, PRECISION_BITS)

parms = seal.EncryptionParameters(seal.SCHEME_TYPE.CKKS)
parms.set_poly_modulus_degree(poly_modulus_degree)
parms.set_coeff_modulus(seal.CoeffModulus.Create(
    poly_modulus_degree, moduli))

context = seal.SEALContext.Create(parms, True, seal.SEC_LEVEL_TYPE.TC128)

We can now generate our keys, the evaluator, encryptor, decryptor and encoder using our context and our keys.

In [3]:
keygen = seal.KeyGenerator(context)

public_key = keygen.public_key()
secret_key = keygen.secret_key()
relin_keys = keygen.relin_keys_local()
galois_keys = keygen.galois_keys_local()

encryptor = seal.Encryptor(context, public_key)
evaluator = seal.Evaluator(context)
decryptor = seal.Decryptor(context, secret_key)
encoder = seal.CKKSEncoder(context)

Just for convenience, we define bellow functions to print directly the content of a ciphertext or plaintext, without having to de decode and decrypt first all the time. This is mainl for educational and debugging purpose.

In [4]:
# export
def print_vector(vec, print_size=4, prec=3):
    """Prints a vector with a given level of precision and print size"""
    slot_count = len(vec)
    print()
    if slot_count <= 2*print_size:
        print("    [", end="")
        for i in range(slot_count):
            print(" " + (f"%.{prec}f" % vec[i]) + ("," if (i != slot_count - 1) else " ]\n"), end="")
    else:
        print("    [", end="")
        for i in range(print_size):
            print(" " + (f"%.{prec}f" % vec[i]) + ",", end="")
        if len(vec) > 2*print_size:
            print(" ...,", end="")
        for i in range(slot_count - print_size, slot_count):
            print(" " + (f"%.{prec}f" % vec[i]) + ("," if (i != slot_count - 1) else " ]\n"), end="")
    print()
    
def print_ptx(ptx: seal.Plaintext):
    result = encoder.decode_double(ptx)
    print_vector(result, 3, 7)
    
def print_ctx(ctx: seal.Ciphertext):
    ptx = seal.Plaintext()
    decryptor.decrypt(ctx, ptx)
    print_ptx(ptx)

We can now start playing with SEAL. Let's see how to encode and encrypt a vector using SEAL.

In [5]:
x = [1,2,3,4]
ptx = seal.Plaintext()
ctx = seal.Ciphertext()

encoder.encode(x, scale, ptx)
encryptor.encrypt(ptx, ctx)

We can print the plaintext and the ciphertext and make sure they output the same thing.

In [6]:
print_ptx(ptx)
print_ctx(ctx)


    [ 1.0000000, 2.0000000, 3.0000000, ..., 0.0000000, -0.0000000, 0.0000000 ]


    [ 1.0000000, 2.0000000, 3.0000000, ..., -0.0000000, -0.0000000, -0.0000000 ]



### Homomorphic operations

#### Addition

We will encode another vector so that we will be able to perform addition and multiplication between them.

In [7]:
y = [-1, -2, -3, -4]
pty = seal.Plaintext()
cty = seal.Ciphertext()

In [8]:
encoder.encode(y, scale, pty)
encryptor.encrypt(pty, cty)

Now let's see how we can add them together : 

In [9]:
ct_plus = seal.Ciphertext()
evaluator.add(ctx, cty, ct_plus)

print_ctx(ct_plus)


    [ 0.0000000, 0.0000000, 0.0000000, ..., -0.0000000, -0.0000000, -0.0000000 ]



Perfect ! It works ! That was pretty easy. 

#### Multiplication

Let's see now how we can multiply them together :

In [10]:
ct_mul = seal.Ciphertext()
evaluator.multiply(ctx, cty, ct_mul)

print_ctx(ct_mul)


    [ -1.0000000, -4.0000000, -9.0000000, ..., 0.0000000, -0.0000000, -0.0000000 ]



This works as well ! 

Nonetheless things have become more complicated with multiplication, because underneath, the number of polynomials needed for decryption, stored in the ciphertext, have increased : 

In [11]:
print(f"Size of the ciphertext after multiplication {ct_mul.size()}")
print(f"Size of the ciphertext after addition {ct_plus.size()}")

Size of the ciphertext after multiplication 3
Size of the ciphertext after addition 2


If we let things go as they are, the size of our ciphertext will keep increasing after each multiplication which will make everything more complicated and error prone.

That's why we will use the relinearization keys defined earlier, in order to reduce the size of the ciphertext : 

In [12]:
print(f"Size of the ciphertext before relinearization {ct_mul.size()}")
evaluator.relinearize_inplace(ct_mul, relin_keys)
print(f"Size of the ciphertext after relinearization {ct_mul.size()}")

Size of the ciphertext before relinearization 3
Size of the ciphertext after relinearization 2


So that's all good ! 

But another problem emerges in CKKS after multiplication : the scale has changed. Because we of the multiplication, we have : $z = \Delta x * \Delta y = \Delta^2 x y$. Therefore, if we add something of a different scale, let's say $\Delta x$, SEAL will throw an error because we try to add things on different scales :

In [13]:
try:
    evaluator.add_inplace(ct_mul, ctx)
except ValueError as e:
    print(e)

scale mismatch


We can check this manually :

In [14]:
print(ct_mul.scale)
print(ctx.scale)

1.2089258196146292e+24
1099511627776.0


So now we need to rescale $z$ by using one of our prime contained in the moduli, which will decrease the level by one : 

In [15]:
evaluator.rescale_to_next_inplace(ct_mul)
print(ct_mul.scale)

1099511775231.0198


We see that our scale has indeed decreased, nonetheless because we divided by a prime close to $\Delta$ and not exactly $\Delta$, we still do not have things of the exact same scale : 

In [16]:
print(ct_mul.scale == scale)
print(ct_mul.scale)
print(scale)

False
1099511775231.0198
1099511627776.0


We can see that the scales are pretty close, but still not the same. That's why we need to manually force the scale of $z$ to be equal to $\Delta$.

In [17]:
ct_mul.scale = scale
print(ct_mul.scale == ctx.scale)

True


Things are not over yet, because we used one prime and threw it away in $z$, we reduced the level of $z$, while $x$ is on another level, because its moduli contains one more prime.

In [18]:
try:
    evaluator.add_inplace(ct_mul, ctx)
except ValueError as e:
    print(e)

encrypted1 and encrypted2 parameter mismatch


So the final thing we need to do is to bring $x$ to the same level as $z$, by removing one of its prime modulus, and add them together : 

In [19]:
ctx_leveled = seal.Ciphertext()
evaluator.mod_switch_to(ctx, ct_mul.parms_id(), ctx_leveled)

try:
    evaluator.add_inplace(ct_mul, ctx_leveled)
except ValueError as e:
    print(e)

In [20]:
print_ctx(ct_mul)


    [ -0.0000001, -2.0000005, -6.0000012, ..., -0.0000000, -0.0000000, 0.0000000 ]



#### Rotation

Finally, we have one more operation in CKKS which is rotation, which allows us to shift the slots of our vectors homomorphically.

In [21]:
print("Initial vector : ")
print_ctx(ctx)

ctx_shifted = seal.Ciphertext()

print("Galois rotation of 1 (shift on the left) :")
evaluator.rotate_vector(ctx, 1, galois_keys, ctx_shifted)
print_ctx(ctx_shifted)

print("Galois rotation of -1 (shift on the right) :")
evaluator.rotate_vector(ctx, -1, galois_keys, ctx_shifted)
print_ctx(ctx_shifted)

Initial vector : 

    [ 1.0000000, 2.0000000, 3.0000000, ..., -0.0000000, -0.0000000, -0.0000000 ]

Galois rotation of 1 (shift on the left) :

    [ 2.0000008, 3.0000001, 4.0000000, ..., -0.0000000, -0.0000000, 1.0000000 ]

Galois rotation of -1 (shift on the right) :

    [ -0.0000000, 1.0000000, 2.0000000, ..., 0.0000000, -0.0000000, 0.0000000 ]



### Example : evaluating the polynomial $P(X) = \pi X^3 + 0.4X + 1$